ResNet50V2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from skimage import exposure

IMPORT AND PREPROCSSING 

In [ ]:
x_test = np.load('/content/drive/MyDrive/school/CS/cs622/skincancer/test_dataset/x_test.npy')
x_train = np.load('/content/drive/MyDrive/school/CS/cs622/skincancer/train_dataset/x_train.npy')
y_train = np.load('/content/drive/MyDrive/school/CS/cs622/skincancer/train_dataset/y_train.npy')[:,0].reshape((2000,1))
y_test = np.load('/content/drive/MyDrive/school/CS/cs622/skincancer/test_dataset/y_test.npy')[:,0].reshape((120,1))

def histogram_equalize(img):
    img = img.reshape((16384,))
    img_cdf, bin_centers = exposure.cumulative_distribution(img)
    return np.interp(img, bin_centers, img_cdf).reshape((128,128,1))
for i in range(x_test.shape[0]):
  x_train[i] = histogram_equalize(x_train[i])
  x_test[i] = histogram_equalize(x_test[i])

x_test = np.concatenate((x_test,)*3, axis=-1)
x_train = np.concatenate((x_train,)*3, axis=-1)

In [ ]:
print("number of training data: ",y_train.shape[0])
print("number of benign in training data: ",np.count_nonzero(y_train))

In [ ]:
kf = StratifiedKFold(n_splits=10,shuffle=True)
print("number of folds: ",kf.get_n_splits(x_train))

In [ ]:
Acc=np.array([])
# 0 malignant 1 benign
for train_index, test_index in kf.split(x_train, y_train):
  tf.keras.backend.clear_session()
  model = tf.keras.applications.resnet_v2.ResNet50V2(include_top = False,input_shape = (128,128,3),classes=2,weights='imagenet')
  for layer in model.layers:
    layer.trainable = False
  x = tf.keras.layers.Flatten()(model.output)
  x = tf.keras.layers.Dense(512, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(256, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  predictions = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

  head_model = tf.keras.Model(inputs = model.input, outputs = predictions)
  head_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])

  
  history = head_model.fit(x_train[train_index], y_train[train_index], batch_size=16, epochs=10, validation_data=(x_train[test_index],y_train[test_index]))
  predic = head_model.predict(x_test)
  predic = np.where(predic > 0.5, 1, 0)
  print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, predic))
  Acc = np.append(Acc,[metrics.accuracy_score(y_test, predic)])
  print('------------------------------------------------------------------------------------------------------------------------------------')
print("AVERAGE TEST SCORE: ",np.average(Acc))

RANDOM FOREST & LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

In [ ]:
x_test = x_test[:,:,:,:1].reshape((120,16384))
x_train = x_train[:,:,:,:1].reshape((2000,16384))

X = np.concatenate((x_train,x_test),axis=0).reshape((2120,16384))

y_train = np.load('/content/drive/MyDrive/school/CS/cs622/skincancer/train_dataset/y_train.npy')[:,0].reshape((2000,))
y_test = np.load('/content/drive/MyDrive/school/CS/cs622/skincancer/test_dataset/y_test.npy')[:,0].reshape((120,))

Y = np.concatenate((y_train,y_test),axis=0).reshape((2120,))


In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 200, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [int(x) for x in np.linspace(1, 100, num = 11)]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
               }

In [ ]:
cl = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = cl, 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 10, verbose=2, random_state=42)
# Fit the random search model
rf_random.fit(x_train, y_train)

In [ ]:
print(rf_random.best_params_)

In [ ]:
print(rf_random.best_score_)

In [ ]:
# Number of trees in random forest
n_e = [int(x) for x in np.linspace(start = 180, stop = 190, num = 6)]
# Number of features to consider at every split
max_f = ['sqrt']
# Maximum number of levels in tree
max_d = [110]
# Minimum number of samples required to split a node
min_ss = [int(x) for x in np.linspace(2, 5, num = 5)]
# Minimum number of samples required at each leaf node
min_sl = [1]
# Method of selecting samples for training each tree
b = ['False']

grid = {'n_estimators': n_e,
               'max_features': max_f,
               'max_depth': max_d,
               'min_samples_split': min_ss,
               'min_samples_leaf': min_sl,
               'bootstrap': b,
               }
cl = RandomForestClassifier()
rf_grid = GridSearchCV(estimator = cl, param_grid = grid,  cv = 10, verbose=3)
# Fit the random search model
rf_grid.fit(x_train, y_train)

In [ ]:
rf_grid.best_params_

In [ ]:
rf_grid.best_score_

In [ ]:
cl = RandomForestClassifier()
y_pred = cl.predict(x_test)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))

Logistic Regression

In [ ]:
C = np.logspace(-4, 4, 20)
solver = ['lbfgs', 'saga']

grid = {'C': C,
        'solver': solver,
        }
cl = LogisticRegression()
LR_grid = GridSearchCV(estimator = cl, param_grid = grid,  cv = 10, verbose=2)
LR_grid.fit(x_train, y_train)

In [ ]:
print(LR_grid.best_params_)

In [ ]:
print(LR_grid.best_score_)